In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time

In [ ]:
form_input = {
    'cbTipoConsulta': 'Repasse',          # Tipo_de_OPC
    'cbAno': '2019',                      # Ano Inicio Vigencia
    'cbPrograma': 'ALIMENTAÇÃO ESCOLAR',  # Programa/Projeto
    'cbUf': 'Bahia'                       # UF
}

filename = 'PNAE'
for value in form_input.values():
    filename += '_' + value.replace(' ','_')
filename += '.csv'

In [ ]:
driver = webdriver.Firefox()

url = 'https://www.fnde.gov.br/sigpcadm/sistema.pu?operation=localizar'

# query_str = '#consulta=tipoConsulta%3D2%26anoConcessao%3D2019%26programaId%3DC7%26ufId%3DBA'

driver.get(url)
time.sleep(5)

In [ ]:
for element_id, element_content in form_input.items():
    print(f'{element_id}: {element_content}')
    element = driver.find_element_by_id(element_id)
    element.send_keys(element_content)
    time.sleep(2)
    element.send_keys(Keys.TAB)

In [ ]:
time.sleep(2)
element_id = 'ext-gen218'  # botão pesquisar
element = driver.find_element_by_id(element_id)
ActionChains(driver).move_to_element(element).click().perform()
time.sleep(5)

In [ ]:
element_id = 'ext-gen240' # tabela com resultados (body)
element = driver.find_element_by_id(element_id)
# element.text
table_content = element.get_attribute('innerHTML')
row_content = table_content.split('Repasse')

In [ ]:
# row_content[1]

In [ ]:
href='javascript:abrirNoPconline("2575915")'
driver.execute_script(href)
alert_obj = driver.switch_to.alert
alert_obj.accept()

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
element_id = 'div_direita'
element = driver.find_element_by_id(element_id)
table_content = element.get_attribute('innerHTML')

In [ ]:
content = element.text
content.split(':')
# content
table_content

from bs4 import BeautifulSoup
soup = BeautifulSoup(table_content)
# soup[0]
# type(soup)
head_cols = []
for head in soup.find_all('th'):
    i = head.text.find('T')
    head_text = head.text.strip(':')
    if head_text:
        head_cols.append(head_text)
#     print(i,head.text,type(head.text))
# print(len(soup.find_all('td')),len())
head_cols

In [ ]:
data_cols = []
for table_data in soup.find_all('td'):
    data = table_data.text.strip(':').strip()
    if data:
        data_cols.append(data)
#     print(i,head.text,type(head.text))
# print(len(soup.find_all('td')),len())
data_cols

{col:data for col,data in zip(head_cols,data_cols)}

In [ ]:
filename

In [ ]:
filename

driver.close()
driver.switch_to.window(driver.window_handles[-1])
driver.close()